In [1]:
import os
from langchain_core.messages import HumanMessage, SystemMessage
from IPython.display import display, Markdown

if not os.environ.get("Deepseek_API_KEY"):
  os.environ["Deepseek_API_KEY"] = getpass.getpass("Enter API key for deepseek: ")
import getpass
from langchain.chat_models import init_chat_model

llm1 = init_chat_model(model="deepseek-chat", model_provider="deepseek")
llm2 = init_chat_model(model="deepseek-chat", model_provider="deepseek")

from langchain_community.embeddings import BaichuanTextEmbeddings
import os

# 从环境变量中获取 API Key
api_key = os.getenv("BAICHUAN_API_KEY")

# 创建 BaichuanTextEmbeddings 实例
embeddings = BaichuanTextEmbeddings(
    baichuan_api_key=api_key,
    base_url="https://api.baichuan-ai.com/v1/embeddings"
)

from langchain_core.vectorstores import InMemoryVectorStore

# 创建向量存储
vector_store = InMemoryVectorStore(embeddings)


In [2]:
from langchain_core.documents import Document

def split_by_model_section_strict(md_path):
    """
    严格按三级标题（###）分块，小标题不会被拆分

    Args:
        md_path (str): Markdown 文件路径

    Returns:
        list of Document: 每个模型块转换为一个 Document 对象
    """
    blocks = []
    current_block = None

    with open(md_path, 'r', encoding='utf-8') as f:
        for line in f:
            # 判断是否是模型块的起始标题
            if line.startswith('### ') and not line.startswith('####'):
                # 如果已经在处理一个块，先保存它
                if current_block:
                    # 创建Document对象并加入blocks列表
                    blocks.append(Document(page_content=current_block['content'], metadata={'title': current_block['title']}))
                # 开始新的块
                current_block = {
                    'title': line.strip('# ').strip(),
                    'content': ''
                }
            else:
                # 把内容加到当前块中（包括小标题）
                if current_block is not None:
                    current_block['content'] += line

        # 添加最后一个块
        if current_block:
            blocks.append(Document(page_content=current_block['content'], metadata={'title': current_block['title']}))

    return blocks

all_splits = split_by_model_section_strict('VRP.md')
for doc in all_splits:
    print(doc.metadata, doc.page_content[:100])  # 打印文档标题和内容的前100个字符
# Index chunks
_ = vector_store.add_documents(documents=all_splits)

{'title': '1. 基本车辆路径问题（VRP）'} 
#### 问题描述：
给定一组客户点、车辆容量、车辆数量、起始点和终点，目标是找到使得所有客户点都被访问一次的最短路径方案。基本车辆路径问题（Vehicle Routing Problem，VRP）
{'title': '2. 带容量约束的车辆路径问题（CVRP）'} 
#### 问题描述：
在CVRP问题中，目标是找到一种车队分配方案，使得每辆车都从一个起始点出发，途经访问每个客户点，并最终返回起始点，同时满足容量约束和最小化车辆的总行驶距离或成本。

#### 
{'title': '3. 带时间窗的车辆路径问题（VRPTW）'} 
#### 问题描述：
在基本VRP的基础上，每个客户点都有一个时间窗，表示可以在某个时间范围内访问。目标是在满足时间窗和车辆容量限制的情况下，最小化总行驶距离或成本。

#### 参数：
- \( 
{'title': '4.钢管切割问题数学模型'} 
#### 问题描述：
某钢管零售商从钢管厂进货，将钢管按照顾客的需求切割后售出。现有客户对不同长度的钢管有需求，目标是最节省地切割19米长的原料钢管，满足顾客的需求，并最小化浪费。

#### 参数


In [3]:
from langchain_core.tools import tool

def retrieve(query: str):
    """Retrieve information related to a query."""
    # 模拟检索到的文档
    retrieved_docs = vector_store.similarity_search(query, k=2)
   
    # 获取前2个相关度最高的文档并获取其中的content
    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)
    # print(docs_content)
    return docs_content, retrieved_docs

In [4]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import MessagesState
from typing import Annotated, Literal


# 假设已经有llm和retrieve工具

# 工具节点，用于调用retrieve函数进行检索
tools = ToolNode([retrieve])

def multi_turn_query(state: MessagesState):
    """多轮问答功能，用于询问并明确用户的需求"""
    
    # 获取最新的用户输入
    latest_message = state["messages"][-1] if state["messages"] else None
    while not latest_message or not isinstance(latest_message, HumanMessage):
        # 如果没有用户输入，返回提示信息
        # 这里可以返回一个提示信息，要求用户提供需求描述
        # Ask the user for input
        user_input = input("请描述您的需求: ")
        # 将用户输入添加到消息中
        state["messages"].append(HumanMessage(user_input))
        latest_message = state["messages"][-1] if state["messages"] else None

    if latest_message:
         # 步骤1: 将用户输入的内容交给LLM进行补充，生成更完整的需求描述
        generate_message = (
            f"你是一个运筹建模需求分析师，负责理解和补充用户的需求描述。\n"
            f"请根据用户提供的需求，简洁、准确地重复并补充其描述。\n"
            f"避免任何发散性的回答，重点放在精确地理解和传达用户的需求，不做额外的解释或推测,也不给出具体数据例子。\n"
            f"反馈的内容应只是文字描述\n"
            f"你输出的格式如下：\n"
            f"根据您的输入，以下是当前的需求总结：\n"
            f"1. 您需要的信息是关于 {latest_message.text}。\n"  # 使用用户的输入作为初步信息\n"
            f"2. 该问题标准模型通常考虑的标准约束：\n"
            f"   - 例如：约束条件可能涉及资源的限制，必须满足的最小或最大值，或者是系统的容量限制等。\n"
            f"3. 该问题标准模型通常考虑的标准目标函数：\n"
            f"   - 例如：目标函数可能是成本最小化、利润最大化、效益最优化等。\n"
            f"4. 该问题标准模型通常考虑的标准变量：\n"
            f"   - 例如：变量可能包括生产数量、时间周期、库存水平等。\n"
            f"5. 用户需求的特殊约束或目标描述：\n"
            f"   - [列出根据用户描述的特殊要求]\n"
            f"6. 其他需要注意的事项：\n"
            f"   - [列出可能需要补充或澄清的部分]\n\n"
            f"7. 提示用户部分需求需要进一步补充和明确：\n"

            "\n\n")
        system_message = SystemMessage(generate_message)
        state["messages"].append(system_message)
        response = llm1.invoke(state["messages"])  # 获取LLM的响应
        state["messages"].append(AIMessage(response.content))
        # 打印LLM的响应
        # print("---AImessage---:\n", response.content)  # 打印LLM的响应
        state["messages"][-1].pretty_print()

    return {"messages":[response]}

def demand_confirm(state: MessagesState)-> Literal["multi_turn_query", "generate"]:
    
    """确认用户的需求描述"""
    while True:
        # Ask the user for input
        user_input = input("你是想继续补充需求还是直接进行建模 (continue/end): ").strip().lower()
        
        # Validate input
        if user_input == "continue":
            # If the user wants to continue, return "multi_turn_query" to proceed with further questioning
            return "multi_turn_query"
        elif user_input == "end":
            # If the user wants to end, return "generate" to proceed with modeling
            return "generate"
        else:
            print("输入无效，请只输入 'continue' 或 'end'")

def generate(state: MessagesState):
    """Retrieve information related to a query."""
    # 获取ToolMessage
  
    recent_demand_messages = []
    for message in reversed(state["messages"]):
        if message.type == "ai":
            recent_demand_messages.append(message)
        else:
            break
    demand_decription_message = recent_demand_messages[::-1]
    docs_content, retrieved_docs = retrieve(demand_decription_message[-1].content)

    system_prompt = (
            f"你是一个运筹建模师，负责理解用户的需求描述并进行建模。\n"
            f"请根据用户提供的需求，调用retrieve外部工具或直接输出相应的数学模型，要求满足用户需求。\n"
            f"请确保检索到的内容与用户需求相关，并且基于检索内容能够输出数学模型。\n"
            f"公式内容按以下要求生成：1. 使用Markdown格式；2. 行间公式用$$包围，行内公式用$包围；\n"
            f"避免发散性的回答以及忽略提示用户的部分，重点放在精确地理解和传达用户的需求，不做额外的解释或推测。\n"
            f"Use the following pieces of retrieved context to answer the question:\n"
            f"{docs_content}"
        )
    
    system_message = SystemMessage(system_prompt)
    state["messages"].append(system_message)
    response = llm1.invoke(state["messages"])  # 获取LLM的响应
    state["messages"].append(AIMessage(response.content))
    # 打印LLM的响应
    # print("---AImessage---:\n", response.content)  # 打印LLM的响应
    # state["messages"][-1].pretty_print()
    display(Markdown(response.content))
    
    return {"messages":[response]}



In [ ]:
from langchain_core.messages import RemoveMessage

def delete_messages(state: MessagesState):
    messages = state["messages"]
    if len(messages) > 2:
        return {"messages": [RemoveMessage(id=m.id) for m in messages[:-2]]}

import json
import numpy as np

def compute_distance_matrix_from_vrp_json(json_str):
    data = json.loads(json_str)

    # 合并 depot 和 customer 的坐标
    locations = [data["depot"]] + data["customers"]
    coords = np.array([[loc["x"], loc["y"]] for loc in locations])

    # 初始化距离矩阵
    n = len(coords)
    dist_matrix = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(coords[i] - coords[j])

    return dist_matrix




def data_input(state: MessagesState):
    """从LLM响应中获取模型描述，并提示用户所需的建模参数"""
    # 获取最近的AI消息
    recent_model_messages = [
        message for message in reversed(state["messages"]) if message.type == "ai"
    ]
    
    # 确保有有效的模型描述
    if recent_model_messages:
        model_description_content = recent_model_messages[0].content
    else:
        model_description_content = "没有找到有效的模型描述，请确认是否已经完成需求分析。"

    # 打印模型描述（用于调试，可以删除或注释掉）
    # print(model_description_content)

    # 创建系统提示信息
    system_prompt = (
          f"你是一个运筹建模师，负责理解用户的需求描述并进行建模。\n"
          f"请根据用户提供的数学模型与需求分析，提示用户需要建模所需要的参数数据，\n"
          f"例如对于VRP问题要明确车辆数量，客户点数量，客户需求。\n"
          f"在VRP类型的问题中，距离矩阵应涵盖“1个起始点 + N个客户点”的所有两两距离，"
          f"因此若有N个客户点，最终的距离矩阵应为(N+1) × (N+1)，即例如10个客户点时，矩阵大小应为11 × 11，距离需保留两位小数。\n"
          f"最终将客户的输入的所有数据整理成json格式，可能会存在多轮补充或更新，你需要整合成最后一个json形式的数据。\n"
          f"如果你认为数据已经补充完整，请告诉用户\"模型已经准备好了，准备开始建立,输入cofirm即可建立\"。\n"
          f"模型描述内容：\n{model_description_content}"
      )

    
    # 创建系统消息并调用LLM
    system_message = SystemMessage(system_prompt)
    state["messages"].append(system_message)
    response = llm1.invoke(state["messages"])
    # 将LLM的响应添加到消息历史中
    state["messages"].append(AIMessage(response.content))
    # 打印LLM的响应（用于调试，可以删除或注释掉）
    state["messages"][-1].pretty_print()

    # user_input = input("请输入所需的的数据（json格式）\n ")
    # # user_input = """{
    # #   "depot": {
    # #     "id": 0,
    # #     "x": 50,
    # #     "y": 50
    # #   },
    # #   "vehicles": [
    # #     { "id": 1, "capacity": 100 },
    # #     { "id": 2, "capacity": 100 },
    # #     { "id": 3, "capacity": 100 },
    # #     { "id": 4, "capacity": 100 }
    # #   ],
    # #   "customers": [
    # #     { "id": 1, "x": 20, "y": 30, "demand": 10 },
    # #     { "id": 2, "x": 25, "y": 60, "demand": 20 },
    # #     { "id": 3, "x": 40, "y": 20, "demand": 15 },
    # #     { "id": 4, "x": 60, "y": 30, "demand": 25 },
    # #     { "id": 5, "x": 70, "y": 70, "demand": 10 },
    # #     { "id": 6, "x": 55, "y": 90, "demand": 30 },
    # #     { "id": 7, "x": 80, "y": 40, "demand": 20 },
    # #     { "id": 8, "x": 30, "y": 80, "demand": 15 },
    # #     { "id": 9, "x": 65, "y": 10, "demand": 25 },
    # #     { "id": 10, "x": 45, "y": 65, "demand": 10 }
    # #   ]
    # # }  """
    # # 计算距离矩阵
    # data = json.loads(user_input)
    while True:
        try:
            user_input = input("请输入所需的数据（json格式）：\n")
            data = json.loads(user_input)
            break
        except json.JSONDecodeError as e:
            print("输入不是合法的 JSON 格式，请重新输入。错误信息：", str(e))
    distance_matrix= compute_distance_matrix_from_vrp_json(user_input)
    data["distance_matrix"] = distance_matrix.tolist()
    json_data = json.dumps(data, indent=2)
    print(json_data)

    while True:
        # 将用户输入添加到消息中
        state["messages"].append(HumanMessage(json_data))
        responsex = llm1.invoke(state["messages"])
        state["messages"].append(AIMessage(responsex.content))
        state["messages"][-1].pretty_print()

        # Ask the user for input
        # user_input = input("请补充所需的的数据,如果数据已经输入完成，请单独输入confirm:\n ")
        user_input = "confirm"
        # 检查用户输入是否为"confirm"
        if user_input.lower() == "confirm":
            break


    return {"messages": [responsex]}


In [6]:
def model_set(state: MessagesState):
    """从AI生成的信息中提取模型需求，生成Gurobi建模代码"""

    # 获取所有 AI 生成的消息内容
    ai_messages = [
        message.content for message in state["messages"] if message.type == "ai"
    ]
    
    # 拼接为完整模型需求描述
    model_description = "\n".join(ai_messages) if ai_messages else "未找到模型需求描述。"
    print("----------------------------这里是模型描述总结-------------------------------------------")
    print(model_description)
    # 系统提示词，要求LLM根据描述生成Gurobi建模代码
    system_prompt = (
        "你是一个熟练的运筹优化专家，擅长使用 Python 中的 Gurobi 进行数学建模。\n"
        "请根据以下模型描述，编写完整的 Gurobi 建模代码，包括变量定义、约束条件、目标函数，并确保代码可执行。\n"
        "注意：VRP问题中需要包括一个不含自环的约束。这个约束可以通过变量定义i!=j中实现，但这样的话，在后续约束中。都需要要求变量i!=j\n"
        # "生成的代码的主要功能是将模型转化为一个lp的模型文件，并存储到指定地址\n"
        # r"C:\Users\86188\Desktop\grade1\AIproduct\LLamchainAIopt\VRP.lp"
        "模型描述如下：\n"
        f"{model_description}"
    )

    # 创建系统提示并调用模型
    system_message = SystemMessage(system_prompt)
    response = llm1.invoke([system_message])

    # 保存并输出生成的代码响应
    generated_code = response.content
    state["messages"].append(AIMessage(generated_code))
    state["messages"][-1].pretty_print()

    return {"messages": [response]}

import re

def run_code(state: MessagesState, filename="generated_model.py"):
    """提取最近AI消息中的Gurobi代码部分，并写入.py文件中"""
    while True:
        # 1. 获取最新的 AI 消息
        recent_code_messages = [
            message for message in reversed(state["messages"]) if message.type == "ai"
        ]

        if not recent_code_messages:
            print("没有找到有效的Gurobi建模代码，请确认是否已经完成建模。")
            return

        # 2. 获取消息内容
        code_content = recent_code_messages[0].content

        # 3. 正则提取 Markdown 代码块（```python ... ``` 或 ``` ... ```)
        code_blocks = re.findall(r"```(?:python)?\n(.*?)```", code_content, re.DOTALL)

        if not code_blocks:
            print("未检测到代码块，请确认内容格式正确。")
            return

        # 4. 拼接所有代码块（如有多个）
        full_code = "\n\n".join(code_blocks)

        
        # 5. 写入 .py 文件
        filename="solve_model.py"
        with open(filename, "w", encoding="utf-8") as f:
            f.write(full_code)

        print(f"Gurobi代码已成功写入：{filename}")

        # 让用户确认是否接受生成的代码
        user_feedback = input("是否接受该代码？输入 'yes' 接受，或输入错误说明进行修改：\n")

        if user_feedback.strip().lower() == "yes":
            break
        else: 
            system_prompt = (
            "你是一个熟练的运筹优化专家，擅长使用 Python 中的 Gurobi 进行数学建模。\n"
            "请根据以下模型描述，编写完整的 Gurobi 建模代码，包括变量定义、约束条件、目标函数，并确保代码可执行。\n"
            "你现在需要根据用户的需求修改原来的代码，\n"
            "原来的代码如下：\n"
            f"{full_code}\n" 
            "用户的反馈如下：\n"
            f"{user_feedback}\n" 
            )
            # 创建系统提示并调用模型
            system_message = SystemMessage(system_prompt)
            response = llm1.invoke([system_message])
            # 保存消息
            # 保存并输出生成的代码响应
            generated_code = response.content
            state["messages"].append(AIMessage(generated_code))
            state["messages"][-1].pretty_print()



In [7]:
from langgraph.graph import MessagesState, StateGraph
from langchain_core.messages import SystemMessage
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver


graph_builder = StateGraph(MessagesState)
from langgraph.graph import END
from langgraph.prebuilt import ToolNode, tools_condition



graph_builder.add_node(multi_turn_query)
graph_builder.add_node(generate)
graph_builder.add_node(data_input)
graph_builder.add_node(model_set)
graph_builder.add_node(delete_messages)
graph_builder.add_node(run_code)

graph_builder.set_entry_point("multi_turn_query")
graph_builder.add_conditional_edges(
    "multi_turn_query",
    demand_confirm,
    # {"multi_turn_query" : "multi_turn_query", "generate" : "generate"}
)
graph_builder.add_edge("generate", 'delete_messages')
graph_builder.add_edge("delete_messages", "data_input")
graph_builder.add_edge("data_input", "model_set")
graph_builder.add_edge("model_set", "run_code")
graph_builder.add_edge("run_code", END)


memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [8]:
config = {"configurable": {"thread_id": "abc123"}}
# input_message = "请帮我建立一个VRP模型，不需要时间窗约束，目标函数是最小化总行驶距离。"
input_message = input("请描述你想要建模的问题: ")
for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    continue
    # step["messages"][-1].pretty_print()

================================== Ai Message ==================================

根据您的输入，以下是当前的需求总结：
1. 您需要的信息是关于某物流公司在一个城市区域内进行快递配送的问题。
2. 该问题标准模型通常考虑的标准约束：
   - 车辆容量限制
   - 客户时间窗口限制
   - 车辆行驶时间限制
   - 仓库运营时间限制
3. 该问题标准模型通常考虑的标准目标函数：
   - 总行驶距离最小化
   - 总配送成本最小化
   - 车辆使用数量最小化
   - 客户满意度最大化
4. 该问题标准模型通常考虑的标准变量：
   - 车辆路线
   - 客户服务顺序
   - 车辆分配
   - 出发/返回时间
5. 用户需求的特殊约束或目标描述：
   - 无特殊约束或目标描述
6. 其他需要注意的事项：
   - 客户地址分布情况未说明
   - 车辆数量是否固定未说明
   - 包裹数量及大小未说明
   - 是否有时间窗要求未说明

7. 提示用户部分需求需要进一步补充和明确：
   - 是否需要考虑客户时间窗要求？
   - 车辆是否有容量限制？
   - 是否有优先配送的客户？
   - 仓库是否有运营时间限制？
================================== Ai Message ==================================

根据您的输入，以下是当前的需求总结：  
1. 您需要的信息是关于某物流公司在一个城市区域内进行快递配送的问题，**且不考虑时间窗约束**。  
2. 该问题标准模型通常考虑的标准约束：  
   - 车辆容量限制（如载重或体积）  
   - 车辆行驶距离或时间限制  
   - 仓库运营时间限制（如车辆需在指定时间内返回）  
3. 该问题标准模型通常考虑的标准目标函数：  
   - 总行驶距离最小化  
   - 总配送成本最小化  
   - 车辆使用数量最小化  
4. 该问题标准模型通常考虑的标准变量：  
   - 车辆路线（路径规划）  
   - 客户服务顺序  
   - 车辆分配（包裹与车辆的匹配）  
5. 用户需求的特殊约束或目标描述：  
   - **明确排

根据您的需求，以下是带容量约束的车辆路径问题（CVRP）的数学模型：

#### 参数：
- \( n \)：客户点数量（仓库编号为0）
- \( m \)：可用车辆数量
- \( Q \)：每辆车的容量上限
- \( d_i \)：客户点\( i \)的包裹需求量（\( d_0 = 0 \)）
- \( c_{ij} \)：从点\( i \)到点\( j \)的行驶成本（距离/时间）

#### 决策变量：
- \( x_{ij}^k \)：二进制变量，车辆\( k \)是否从\( i \)行驶到\( j \)
- \( u_i \)：辅助变量，表示客户点\( i \)的访问顺序（用于消除子环）

#### 目标函数（最小化总成本）：
$$
\min \sum_{k=1}^{m} \sum_{i=0}^{n} \sum_{j=0}^{n} c_{ij} x_{ij}^k
$$

#### 约束条件：
1. **每个客户点仅被访问一次**：
   $$
   \sum_{k=1}^{m} \sum_{j=0}^{n} x_{ij}^k = 1, \quad \forall i \in \{1,...,n\}
   $$

2. **车辆从仓库出发并返回**：
   $$
   \sum_{j=1}^{n} x_{0j}^k = 1, \quad \forall k \in \{1,...,m\}
   $$
   $$
   \sum_{i=1}^{n} x_{i0}^k = 1, \quad \forall k \in \{1,...,m\}
   $$

3. **流量守恒**：
   $$
   \sum_{i=0}^{n} x_{ih}^k = \sum_{j=0}^{n} x_{hj}^k, \quad \forall h \in \{1,...,n\}, \forall k \in \{1,...,m\}
   $$

4. **容量约束**：
   $$
   \sum_{i=1}^{n} d_i \left( \sum_{j=0}^{n} x_{ij}^k \right) \leq Q, \quad \forall k \in \{1,...,m\}
   $$

5. **MTZ子环消除**：
   $$
   u_i - u_j + n x_{ij}^k \leq n-1, \quad \forall i,j \in \{1,...,n\}, i \neq j, \forall k \in \{1,...,m\}
   $$
   $$
   u_i \geq 0, \quad \forall i \in \{1,...,n\}
   $$

6. **二进制约束**：
   $$
   x_{ij}^k \in \{0,1\}, \quad \forall i,j \in \{0,...,n\}, \forall k \in \{1,...,m\}
   $$

注：若车辆数量需优化，可引入额外变量和约束调整模型。

================================== Ai Message ==================================

根据您提供的CVRP模型描述，我需要以下参数数据来建立完整的模型：

1. 客户点数量（n）
2. 可用车辆数量（m） 
3. 每辆车的容量上限（Q）
4. 各客户点的包裹需求量列表（d_i，其中d_0=0）
5. 距离/成本矩阵（c_ij），大小为(n+1)×(n+1)

请提供以下具体数据：

1. 客户点数量n（不包括仓库）：
2. 可用车辆数量m：
3. 每辆车的容量上限Q：
4. 各客户点需求量（请按顺序列出d_1到d_n）：
5. 距离矩阵（请提供完整的(n+1)×(n+1)矩阵）：

例如，如果有2个客户点，数据格式可能如下：
```json
{
  "n": 2,
  "m": 3,
  "Q": 100,
  "demands": [25, 35],
  "distance_matrix": [
    [0, 10.5, 8.2],
    [10.5, 0, 15.3],
    [8.2, 15.3, 0]
  ]
}
```

请提供上述数据，我将帮您整理成完整的JSON格式。当所有数据都齐全后，我会通知您模型已准备好建立。


JSONDecodeError: Expecting value: line 1 column 1 (char 0)